# Forward Error Correction Encoding

In [13]:
from pynq import allocate
from pynq import Overlay
import numpy as np
import xsdfec

ol = Overlay('bitstream/ldpc_loopback.bit')

In [14]:
fec = ol.ldpc_encoder.sd_fec
ldpc_params = fec.available_ldpc_params()
ldpc_params

['docsis_short',
 'docsis_medium',
 'docsis_long',
 'docsis_init_ranging',
 'docsis_fine_ranging',
 'wifi802_11_cr1_2_648',
 'wifi802_11_cr1_2_1296',
 'wifi802_11_cr1_2_1944',
 'wifi802_11_cr2_3_648',
 'wifi802_11_cr2_3_1296',
 'wifi802_11_cr2_3_1944',
 'wifi802_11_cr3_4_648',
 'wifi802_11_cr3_4_1296',
 'wifi802_11_cr3_4_1944',
 'wifi802_11_cr5_6_648',
 'wifi802_11_cr5_6_1296',
 'wifi802_11_cr5_6_1944',
 '5g_graph1_set1_l5_p32',
 '5g_graph1_set1_l5_p64',
 '5g_graph1_set1_l5_p128',
 '5g_graph1_set1_l5_p256',
 '5g_graph1_set2_l5_p384',
 '5g_graph1_set1_l46_p32',
 '5g_graph1_set1_l46_p64',
 '5g_graph1_set1_l46_p128',
 '5g_graph1_set1_l46_p256',
 '5g_graph1_set2_l46_p384',
 '5g_graph2_set1_l7_p32',
 '5g_graph2_set1_l7_p64',
 '5g_graph2_set1_l7_p128',
 '5g_graph2_set1_l7_p256',
 '5g_graph2_set2_l7_p384',
 '5g_graph2_set1_l42_p32',
 '5g_graph2_set1_l42_p64',
 '5g_graph2_set1_l42_p128',
 '5g_graph2_set1_l42_p256',
 '5g_graph2_set2_l42_p384']

In [15]:
fec.CORE_ORDER = 0                         # In order outputs
fec.CORE_AXIS_ENABLE = 0                   # Ensure FEC is disabled (000000)

sc_offset = 0
la_offset = 0
qc_offset = 0
print("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format('Status','Code ID','SC Offset','LA Offset','QC Offset','Code Name'))
print("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format('------','------','------','------','------','------'))
for code_id in range(len(ldpc_params)):
    
    code_name = ldpc_params[code_id]
    sc_table = fec._code_params.ldpc[code_name]['sc_table']
    la_table = fec._code_params.ldpc[code_name]['la_table']
    qc_table = fec._code_params.ldpc[code_name]['qc_table']
    if type(sc_table) != list : sc_table = [sc_table]

    if (sc_offset + len(sc_table)) > len(fec.LDPC_SC_TABLE):
        print("\nNot enough space for SC table: %s. Terminating.\n" % (code_name))
        break
    if (la_offset*4 + len(la_table)) > len(fec.LDPC_LA_TABLE):
        print("\nNot enough space for LA table: %s. Terminating.\n" % (code_name))
        break
    if (qc_offset*4 + len(qc_table)) > len(fec.LDPC_QC_TABLE):
        print("\nNot enough space for QC table: %s. Terminating.\n" % (code_name))
        break
    
    print("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format('Loaded',code_id, sc_offset, la_offset, qc_offset, code_name))
    fec.add_ldpc_params(code_id, sc_offset, la_offset, qc_offset, code_name)

    sc_offset += len(sc_table)
    la_offset += int(np.ceil(len(la_table)/4))
    qc_offset += int(np.ceil(len(qc_table)/4))
    
fec.CORE_AXIS_ENABLE = 63                  # Enable FEC (111111)

Status     Code ID    SC Offset  LA Offset  QC Offset  Code Name 
------     ------     ------     ------     ------     ------    
Loaded     0          0          0          0          docsis_short
Loaded     1          2          2          21         docsis_medium
Loaded     2          4          4          54         docsis_long
Loaded     3          6          6          97         docsis_init_ranging
Loaded     4          8          8          112        docsis_fine_ranging
Loaded     5          9          9          120        wifi802_11_cr1_2_648
Loaded     6          15         15         196        wifi802_11_cr1_2_1296
Loaded     7          21         21         240        wifi802_11_cr1_2_1944
Loaded     8          27         27         270        wifi802_11_cr2_3_648
Loaded     9          31         31         324        wifi802_11_cr2_3_1296
Loaded     10         35         35         360        wifi802_11_cr2_3_1944
Loaded     11         39         39         388       

In [16]:
code_name = 'docsis_short'
code_id = ldpc_params.index(code_name)

n = fec._code_params.ldpc[code_name]['n']
k = fec._code_params.ldpc[code_name]['k']
p = fec._code_params.ldpc[code_name]['p']

print('Block Length (bits): %s\nInformation Bits: %s\nSub-Matrix Size: %s' % (n, k, p))

Block Length (bits): 1120
Information Bits: 840
Sub-Matrix Size: 56


In [17]:
axis_width = 32 // 8
num_words = 1
pad = axis_width // num_words

K = int(np.ceil(k/8))
N = int(np.ceil(n/8))

tx_buffer = allocate(shape=(K*pad,), dtype=np.uint8)
rx_buffer = allocate(shape=(N*pad,), dtype=np.uint8)

for i in range(len(tx_buffer))[::pad]:
    tx_buffer[i] = np.random.randint(1,255)

In [18]:
def non_5G_ldpc_ctrl_enc(id, code):
    id = '{0:08b}'.format(id)
    reserved = '{0:017b}'.format(0)
    code = '{0:07b}'.format(code)
    
    print('Code ID: ', int(id,2))
    print('Code Name: ', int(code,2))
    
    ctrl = id + reserved + code
    
    return(int(ctrl,2))

ctrl_buffer = allocate(shape=(1,), dtype=np.uint32)

ctrl_buffer[0] = non_5G_ldpc_ctrl_enc(code_id, 0)

Code ID:  0
Code Name:  0


In [19]:
def non_5G_ldpc_status_enc(status):
    binary = '{0:032b}'.format(status)
    
    id = binary[0:8]
    hard_op = binary[17]
    op = binary[18]
    code = binary[24:32]

    print('Code ID: ', int(id,2))
    print('Hard Output: ', int(hard_op,2))
    print('Operation: ', int(op,2))
    print('Code Name: ', int(code,2))
    
status_buffer = allocate(shape=(1,), dtype=np.uint32)

In [20]:
dma_data = ol.ldpc_encoder.axi_dma_data
dma_ctrl = ol.ldpc_encoder.axi_dma_ctrl

dma_ctrl.sendchannel.transfer(ctrl_buffer)
dma_data.sendchannel.transfer(tx_buffer)
dma_ctrl.recvchannel.transfer(status_buffer)
dma_data.recvchannel.transfer(rx_buffer)

dma_ctrl.sendchannel.wait()
dma_data.sendchannel.wait()
dma_data.recvchannel.wait()
dma_ctrl.recvchannel.wait()

In [21]:
import pandas as pd
pd.set_option('display.max_rows', 10000)

print('\nStatus - ')
non_5G_ldpc_status_enc(status_buffer[0])

tx_frame = pd.DataFrame({'TX': tx_buffer[::pad]})
rx_frame = pd.DataFrame({'RX': rx_buffer[::pad]})

compare = pd.concat([tx_frame, rx_frame], axis=1)

if not (tx_buffer[0:(K-1)*pad] == rx_buffer[0:(K-1)*pad]).all():
    print('\nFalse: ', code_name, np.asarray(np.where(tx_buffer[0:(K-1)*pad] != rx_buffer[0:(K-1)*pad]))/pad)
else:
    print('\nTrue: ', code_name)


Status - 
Code ID:  0
Hard Output:  1
Operation:  1
Code Name:  0

True:  docsis_short


In [22]:
%store tx_buffer rx_buffer pad

Stored 'tx_buffer' (PynqBuffer)
Stored 'rx_buffer' (PynqBuffer)
Stored 'pad' (int)
